In [ ]:
#media pipeを用いて表情を判定するプログラム

In [ ]:
#うれしい...口角が上がる、目が細くなる、目じりが下がる、口が横に広がる、目じりにしわが寄る、頬が上がる
#好き...口角が上がる、目が細くなる、目じりが下がる...削除
#悲しい...口角が下がる、目じりが下がる、上瞼が下がる、頬が下がる、眉間にしわが寄る
#疲れた...口角が下がる,上瞼が下がる、目が細くなる、眉が下がる、頬が下がる
#びっくり...目が開く、眉が上がる、口が開く
#嫌い...目じりが上がる、眉間が狭くなる、口角が下がる、唇を閉じる、唇をゆがめる

#組むべきコード...口角を調べるコード、目の開き具合を調べるコード、目尻の角度を調べるコード、口を開いているかを調べるコード、口が横に広がっているかを調べるコード、頬が上がっているかを調べるコード
#...表情を判定するコード...←カメラで読み取った表情の特徴一つにつき%を上げていき、一番%が高いものを表示する
#...%を合算する関数と%が最も高い表情を表示する関数を作る

In [1]:
import cv2
import mediapipe as mp
import math

#MediaPipeのセットアップ
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

#2点間の距離を計算
def calculate_distance(point1, point2):
    return math.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

#ベクトルの角度を計算する関数
def calculate_angle(point1, point2):
    # y軸が下方向に増加する画像座標系を考慮
    angle = math.degrees(math.atan2(point2[1] - point1[1], point2[0] - point1[0]))
    return angle

#口角を調べる関数
def mouth_angle_func(landmarks, image_width, image_height):

    #ランドマークの座標を取得
    left_corner = (landmarks[61].x * image_width, landmarks[61].y * image_height)
    right_corner = (landmarks[291].x * image_width, landmarks[291].y * image_height)
    upper_lip = (landmarks[13].x * image_width, landmarks[13].y * image_height)
    
    #口角の角度を計算
    left_angle = calculate_angle(left_corner, upper_lip)
    right_angle = calculate_angle(right_corner, upper_lip)

    #口角の角度で表情を分類
    if left_angle > 10 and right_angle > 10:
        mouth_angle = 1
        return mouth_angle
    elif left_angle < -10 and right_angle < -10:
        mouth_angle = 0
        return mouth_angle

#口の幅と開き具合を調べる関数
def mouth_open_func(landmarks, image_width, image_height):
    #ランドマークの座標を取得
    mouth_left = (landmarks[61].x * image_width, landmarks[61].y * image_height)
    mouth_right = (landmarks[291].x * image_width, landmarks[291].y * image_height)
    mouth_top = (landmarks[13].x * image_width, landmarks[13].y * image_height)
    mouth_bottom = (landmarks[14].x * image_width, landmarks[14].y * image_height)

    #口の形状を計算
    mouth_width = calculate_distance(mouth_left, mouth_right)
    mouth_height = calculate_distance(mouth_top, mouth_bottom)

    #口の状態で表情を分類
    if mouth_height > 5:
        mouth_open = 1
        return mouth_open
    else:
        mouth_open = 0
        return mouth_open

#目が開いているかを調べる関数
def eye_open_func(landmarks, image_width, image_height):
    #ランドマークの座標を取得
    left_eye_top = (landmarks[159].x * image_width, landmarks[159].y * image_height)
    left_eye_bottom = (landmarks[145].x * image_width, landmarks[145].y * image_height)
    right_eye_top = (landmarks[386].x * image_width, landmarks[386].y * image_height)
    right_eye_bottom = (landmarks[374].x * image_width, landmarks[374].y * image_height)

    #目の形状を計算
    left_eye_openness = calculate_distance(left_eye_top, left_eye_bottom)
    right_eye_openness = calculate_distance(right_eye_top, right_eye_bottom)

    #目の状態で表情を分類
    if left_eye_openness > 5 and right_eye_openness > 5:
        eyes_open =  1
        return eyes_open
    else:
        eyes_open =  0
        return eyes_open

#目尻の角度を調べる関数
def eye_angle_func(landmarks, image_width, image_height):
    #ランドマークの座標を取得
    left_corner = (landmarks[33].x * image_width, landmarks[33].y * image_height)
    right_corner = (landmarks[362].x * image_width, landmarks[362].y * image_height)
    left_eye_base = (landmarks[133].x * image_width, landmarks[133].y * image_height)
    right_eye_base = (landmarks[263].x * image_width, landmarks[263].y * image_height)

    #目尻が基準点より上にあるか判定
    left_eye_angle = 1 if left_corner[1] < left_eye_base[1] else 0
    right_eye_angle = 1 if right_corner[1] < right_eye_base[1] else 0

    return left_eye_angle, right_eye_angle

#頬が上がっているかを調べる関数
def cheek_func(landmarks, image_width, image_height):
    left_cheek = (landmarks[234].x * image_width, landmarks[234].y * image_height)
    right_cheek = (landmarks[93].x * image_width, landmarks[93].y * image_height)
    left_eye_bottom = (landmarks[145].x * image_width, landmarks[145].y * image_height)
    right_eye_bottom = (landmarks[374].x * image_width, landmarks[374].y * image_height)

    #左右の頬が目の下に対して上がっているか判定
    left_cheek_status = 1 if left_cheek[1] < left_eye_bottom[1] else 0
    right_cheek_status = 1 if right_cheek[1] < right_eye_bottom[1] else 0

    return left_cheek_status, right_cheek_status
    
#眉間が寄っているかを判定する関数
def brow_dist_func(landmarks, image_width, image_height):
    #ランドマーク座標を取得
    left_brow = (landmarks[70].x * image_width, landmarks[70].y * image_height)
    right_brow = (landmarks[300].x * image_width, landmarks[300].y * image_height)
    left_eye = (landmarks[33].x * image_width, landmarks[33].y * image_height)
    right_eye = (landmarks[263].x * image_width, landmarks[263].y * image_height)

    #左右の眉毛間の距離を計算
    brow_distance = calculate_distance(left_brow, right_brow)


    #顔全体の幅を基準として距離を正規化
    face_width = calculate_distance(left_eye, right_eye)
    normalized_brow_distance = brow_distance / face_width

    #眉間が寄っているかを判定
    if normalized_brow_distance < 0.15:  # 閾値0.15は目安
        brow_dist = 1
        return brow_dist
    else:
        brow_dist = 0
        return brow_dist

#眉の上下を判定する関数
def brow_position_func(landmarks, image_width, image_height):
    #ランドマーク座標を取得
    left_brow = (landmarks[105].x * image_width, landmarks[105].y * image_height)
    right_brow = (landmarks[334].x * image_width, landmarks[334].y * image_height)
    left_eye_top = (landmarks[159].x * image_width, landmarks[159].y * image_height)
    right_eye_top = (landmarks[386].x * image_width, landmarks[386].y * image_height)
    amount = (landmarks[10].x * image_width, landmarks[10].y * image_height)
    jaw = (landmarks[152].x * image_width, landmarks[152].y * image_height)

    #目と眉の距離を計算
    brow_distance_left = abs(left_brow[1] - left_eye_top[1])
    brow_distance_right = abs(right_brow[1] - right_eye_top[1])

    #顎から額までの距離を計算
    face_height = abs(amount[1] - jaw[1])

    #左右の眉毛が基準点より上か下かを判定
    left_brow_height = 1 if brow_distance_left > 0.03 * face_height else 0
    right_brow_height = 1 if brow_distance_right > 0.03 * face_height else 0

    return left_brow_height, right_brow_height


#結果(各表情のリスト)
l = {"joy_face": 0, "sad_face": 0, "tir_face": 0, "sup_face": 0, "hate_face":0}


#カメラ入力
cap = cv2.VideoCapture(0)
with mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
    while cap.isOpened():
        l = {"joy_face": 0, "sad_face": 0, "tir_face": 0, "sup_face": 0, "hate_face":0}
        ret, frame = cap.read()
        if not ret:
            break

        #BGR画像をRGBに変換
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(frame_rgb)

        #ランドマークの描画と表情判定
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                # ランドマークを取得
                landmarks = face_landmarks.landmark
                image_height, image_width, _ = frame.shape
                

                #表情を分類
                #expression = classify_expression(landmarks, image_width, image_height)

                #表情の分類
                
                #口角による分類
                mouth_angle = mouth_angle_func(landmarks, image_width, image_height)
                if mouth_angle == 1:
                    l["joy_face"] = l["joy_face"] + 2
                else:
                    l["sad_face"] = l["sad_face"] + 25
                    l["tir_face"] = l["tir_face"] + 25
                    l["hate_face"] =  l["hate_face"] + 25
                #口の開閉による分類
                mouth_open = mouth_open_func(landmarks, image_width, image_height)
                if mouth_open == 1:
                    l["sup_face"] = l["sup_face"] + 33
                else:
                    l["hate_face"] = l["hate_face"] + 25
                #目の開き度合による分類
                eyes_open = eye_open_func(landmarks, image_width, image_height)
                if eyes_open == 1:
                    l["sup_face"] = l["sup_face"] + 33
                else:
                    l["joy_face"] = l["joy_face"] + 25 
                    l["tir_face"] = l["tir_face"] + 25
                #目尻の傾きによる分類
                left_eye_angle, right_eye_angle = eye_angle_func(landmarks, image_width, image_height)
                if left_eye_angle + right_eye_angle == 2:
                    l["hate_face"] = l["hate_face"] + 25
                else:
                    l["joy_face"] = l["joy_face"] + 25
                    l["sad_face"] = l["sad_face"] + 25
                #頬の上がり具合による分類
                left_cheek_status, right_cheek_status = cheek_func(landmarks, image_width, image_height)
                if left_cheek_status + right_cheek_status == 2:
                    l["joy_face"] = l["joy_face"] + 25
                else:
                    l["sad_face"] = l["sad_face"] + 25
                    l["tir_face"] = l["tir_face"] + 25
                #眉間の幅による分類
                brow_dist = brow_dist_func(landmarks, image_width, image_height)
                if brow_dist == 1:
                    l["sad_face"] = l["sad_face"] + 25
                    l["hate_face"] = l["hate_face"] + 25
                #眉の上下による分類
                left_brow_height, right_brow_height = brow_position_func(landmarks, image_width, image_height)
                if left_brow_height + right_brow_height == 2:
                    l["sup_face"] = l["sup_face"] + 33
                else:
                    l["tir_face"] = l["tir_face"] + 25

                #表情の判定
                

                #結果を表示
                cv2.putText(frame, max(l, key = l.get), (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

                #ランドマークの描画
                mp_drawing.draw_landmarks(
                    image=frame,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_TESSELATION,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1, circle_radius=1)
                )

        #フレームを表示
        cv2.imshow('Expression Detection', frame)

        if cv2.waitKey(5) & 0xFF == 27:  # ESCキーで終了
            break

cap.release()
cv2.destroyAllWindows()


ImportError: DLL load failed while importing _framework_bindings: ダイナミック リンク ライブラリ (DLL) 初期化ルーチンの実行に失敗しました。

In [6]:
import cv2
import mediapipe as mp
#import math

ImportError: DLL load failed while importing _framework_bindings: ダイナミック リンク ライブラリ (DLL) 初期化ルーチンの実行に失敗しました。